# Music Recommendation System

**This is a Machine Learning model to suggest songs to user based on content similarity approach**

#### Importing Libraries and Dataset

In [8]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
songs = pd.read_csv("songdata.csv")
print(songs.shape)
songs.head()

(57650, 4)


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


#### Data cleaning

In [5]:
df = songs.sample(n=5000).drop('link',axis=1).reset_index(drop=True)
print(df.shape)
df.head()

(5000, 3)


,artist,song,text
0,Dolly Parton,I Dreamed Of A Hillbilly Heaven,I dreamed I was there in hillbilly heaven oh w...
1,Styx,What Has Come Between Us,Look at all the people \nLooking at their dre...
2,Bon Jovi,Bed Of Roses,Sitting here wasted and wounded \nAt this old...
3,Chris Rea,"Square Peg, Round Hole",He's a square peg in a round hole \nMisfit in...
4,Amy Grant,It's The Most Wonderful Time Of The Year,It's the most wonderful time of the year \n ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  5000 non-null   object
 1   song    5000 non-null   object
 2   text    5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


In [7]:
df.isnull().sum()

artist    0
song      0
text      0
dtype: int64

In [9]:
df['artist'] = df['artist'].apply(lambda x:x.replace(" ",""))
df.head()

,artist,song,text
0,DollyParton,I Dreamed Of A Hillbilly Heaven,I dreamed I was there in hillbilly heaven oh w...
1,Styx,What Has Come Between Us,Look at all the people \nLooking at their dre...
2,BonJovi,Bed Of Roses,Sitting here wasted and wounded \nAt this old...
3,ChrisRea,"Square Peg, Round Hole",He's a square peg in a round hole \nMisfit in...
4,AmyGrant,It's The Most Wonderful Time Of The Year,It's the most wonderful time of the year \n ...


In [10]:
df['text'][0]

"I dreamed I was there in hillbilly heaven oh what a beautiful sight  \nAnd I met all the stars in hillbilly heaven oh what a star spangled night  \nLast night I dreamed I went to hillbilly heaven  \nAnd just as I arrived the old curtain lifted  \nAnd there on stage stood who else but Roy Acuff and Tex Ritter  \nThey said well how you doing come on in here  \nI know there's some folks you want to see  \nOver there's our newest member your old friend Conway Twitty say Hello Darlin'  \nAnd there's gentlemen Jim Reeves singin' with heaven's choir  \nRoger Miller he just got here he's making everybody laugh  \nCourse you know Roy Orbison can sing just as high as anybody here  \nAnd Ernest Tubb can sing as low  \nAnd Patsy Cline well bet she's the best singin' angel here  \nAnd Dottie West is probably the prettiest  \nThere's the old pea-picker Ernie Ford  \nJimmy Rodgers George Morgan Hank Williams  \nAnd the wandering boy Web Pierce  \nThe old honky tonker himself Lefty Frizzell  \nMy goo

In [14]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n','', regex=True)

df['text'][0]

"i dreamed i was there in hillbilly heaven oh what a beautiful sight  and i met all the stars in hillbilly heaven oh what a star spangled night  last night i dreamed i went to hillbilly heaven  and just as i arrived the old curtain lifted  and there on stage stood who else but roy acuff and tex ritter  they said well how you doing come on in here  i know there's some folks you want to see  over there's our newest member your old friend conway twitty say hello darlin'  and there's gentlemen jim reeves singin' with heaven's choir  roger miller he just got here he's making everybody laugh  course you know roy orbison can sing just as high as anybody here  and ernest tubb can sing as low  and patsy cline well bet she's the best singin' angel here  and dottie west is probably the prettiest  there's the old pea-picker ernie ford  jimmy rodgers george morgan hank williams  and the wandering boy web pierce  the old honky tonker himself lefty frizzell  my goodness what a group  would you just l

In [15]:
df['tags'] = df['artist'] + df['text']
df.head()

,artist,song,text,tags
0,DollyParton,I Dreamed Of A Hillbilly Heaven,i dreamed i was there in hillbilly heaven oh w...,DollyPartoni dreamed i was there in hillbilly ...
1,Styx,What Has Come Between Us,look at all the people looking at their dream...,Styxlook at all the people looking at their d...
2,BonJovi,Bed Of Roses,sitting here wasted and wounded at this old p...,BonJovisitting here wasted and wounded at thi...
3,ChrisRea,"Square Peg, Round Hole",he's a square peg in a round hole misfit in a...,ChrisReahe's a square peg in a round hole mis...
4,AmyGrant,It's The Most Wonderful Time Of The Year,it's the most wonderful time of the year it...,AmyGrantit's the most wonderful time of the ye...


In [16]:
new_df = df[['song','tags']]
new_df.head()

,song,tags
0,I Dreamed Of A Hillbilly Heaven,DollyPartoni dreamed i was there in hillbilly ...
1,What Has Come Between Us,Styxlook at all the people looking at their d...
2,Bed Of Roses,BonJovisitting here wasted and wounded at thi...
3,"Square Peg, Round Hole",ChrisReahe's a square peg in a round hole mis...
4,It's The Most Wonderful Time Of The Year,AmyGrantit's the most wonderful time of the ye...


### Preprocessing

In [17]:
# We need to tokenize the similar words in the tags columns [love,loving] ->[love]
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [18]:
def stem(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)

In [19]:
new_df['tags'] = new_df['tags'].apply(stem)

**Tags vectorization to compare similarity**

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tf.fit_transform(new_df['tags'])

**Importing cosine similarity library to compare the similarity with each other**

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(matrix)
print(similarity.shape)
similarity[0]

(5000, 5000)


array([1.        , 0.01027342, 0.04526345, ..., 0.03166247, 0.03881116,
       0.08704583])

### Recommendation function

In [25]:
def recommendation(song_df):
    idx = new_df[new_df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [26]:
recommendation("Bed Of Roses")

['Bed In Summer',
 'Time For Bed',
 'Lay It Down',
 "I'd Just Love To Lay You Down - Conway Twitty",
 'Give My Love To Rose',
 'Lay Down Beside Me',
 'A Last Request',
 'A Teardrop On A Rose',
 'Loving You',
 'Too Close To Heaven',
 'Summer In December',
 'Give It All We Got Tonight',
 "I'll Never Get Enough Of You",
 "Don't Try To Close A Rose",
 "I Can't Keep This From You",
 "I'm Your Baby Tonight",
 "No One's Gonna Sleep Tonight",
 'Love Me Tonight',
 'I Just Want You',
 'House']

In [28]:
recommendation('Square Peg, Round Hole')

['Hole Hearted',
 'Still Falls The Rain',
 'Incarceration Of A Flower Child',
 'Cool Love',
 'This Song',
 'Black Hole Sun',
 'House',
 'Jimmy Newman',
 'Clockwork Angels',
 'Soldier In Our Town',
 'On Your Own',
 'Hole In Your Soul',
 'Infinity',
 'Five Years',
 '7 Years',
 'Heading Out To The Highway',
 'Sun Come Up',
 'Abacab',
 'Little Miss Fit',
 'This Land Is Your Land']